In [1]:
import pandas as pd
import sqlite3 as sl
import uuid
import datetime as dt

In [11]:
uuid.uuid4()

UUID('a395e2e4-f3f5-4f20-8046-a189bcd8a27e')

In [2]:
con = sl.connect("mydata.db")

In [33]:
con.execute("DROP TABLE TRADE_MAIN")

con.execute("""
    CREATE TABLE TRADE_MAIN(
        id INTEGER PRIMARY KEY,
        trade_group_id TEXT,
        entry_date REAL,
        ticker TEXT,
        stk1 REAL,
        stk1_type INTEGER,
        stk1_exp REAL,
        stk2 REAL,
        stk2_type INTEGER,
        stk2_exp REAL,
        stk3 REAL,
        stk3_type INTEGER,
        stk3_exp REAL,
        stk4 REAL,
        stk4_type INTEGER,
        stk4_exp REAL,
        cost REAL,
        trdprice REAL,
        BRP REAL,
        strategy REAL
    )
""")

In [3]:
# con.execute("DROP TABLE TRADE_SUMMARY")

con.execute("""
    CREATE TABLE TRADE_SUMMARY(
        id INTEGER PRIMARY KEY,
        entry_date REAL,
        exit_date REAL,
        trade_length INTEGER,
        ticker TEXT,
        stk_width REAL,
        pnl REAL,
        strategy TEXT
    )
""")

In [15]:
con.execute("DROP TABLE TRADE_HIST")

con.execute("""
    CREATE TABLE TRADE_HIST(
        id INTEGER PRIMARY KEY,
        trade_group_id TEXT,
        entry_date REAL,
        ticker TEXT,
        stk1 REAL,
        stk1_type INTEGER,
        stk1_exp REAL,
        stk2 REAL,
        stk2_type INTEGER,
        stk2_exp REAL,
        stk3 REAL,
        stk3_type INTEGER,
        stk3_exp REAL,
        stk4 REAL,
        stk4_type INTEGER,
        stk4_exp REAL,
        cost REAL,
        trdprice REAL,
        BRP REAL,
        strategy REAL
    )
""")

In [9]:
def enter(new_entryl):
    sql = """
        INSERT INTO TRADE_MAIN(
        trade_group_id, 
        entry_date,
        ticker,
        stk1 ,
        stk1_type,
        stk1_exp,
        stk2,
        stk2_type,
        stk2_exp,
        cost,
        trdprice,
        BRP 
        ) VALUES (?, julianday(?),?,?,?,julianday(?),?,?,julianday(?),?,?,?)

    """
    with con:
        con.executemany(sql,new_entryl)

In [10]:
def get_stats(uuid):
    data = con.execute(f"""
    SELECT ticker, date(entry_date), sum(trdprice), sum(cost), stk1, stk2 FROM TRADE_MAIN
    GROUP BY trade_group_id = '{uuid}'
""")

    for i in data:
        ticker = i[0]
        exp_date = dt.datetime.strptime(i[1], '%Y-%m-%d').date()
        dte = (exp_date - dt.date.today()).days
        bes = round(i[-1] - i[2],1)
        bem = i[2]
        bema = round(i[3]/100,2)
        m50 = round(i[2]/2,2)
        cost = round(i[-3],2)
        print(ticker, dte, bes, bem, bema, m50, cost)

In [11]:
def create_new_entry(entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice):
    entry = [
        (str(uuid.uuid4()), entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice, (((stk2-stk1)*100)-cost))
    ]

    return entry

In [12]:
def update_trade(uuid, entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice):
    """
    This function is to help you prepare trade information in the right format for writing to the database
    """
    entry = [
    (uuid, entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice, (((stk2-stk1)*100)-cost))
]
    return entry

In [ ]:
records = [(1, 'Glen', 8),
			(2, 'Elliot', 9),
			(3, 'Bob', 7)]


In [13]:
def close(close_entry):

    uuid = close_entry[0][0]

    sql = """
        INSERT INTO TRADE_MAIN(
        trade_group_id, 
        entry_date,
        ticker,
        stk1 ,
        stk1_type,
        stk1_exp,
        stk2,
        stk2_type,
        stk2_exp,
        cost,
        trdprice,
        BRP 
        ) VALUES (?, julianday(?),?,?,?,julianday(?),?,?,julianday(?),?,?,?)

    """
    with con:
        con.executemany(sql,close_entry)

    data = con.execute(f"""
    SELECT ticker, max(date(entry_date)), min(date(entry_date)), sum(trdprice), sum(cost), min(stk1), stk2 FROM TRADE_MAIN
    WHERE trade_group_id = '{uuid}'
    """)

    for i in data:
        ticker = i[0]
        exit_date = str(i[1])
        entry_date = str(i[2])
        days_in_trade = (dt.datetime.strptime(exit_date, '%Y-%m-%d').date() - dt.datetime.strptime(entry_date, '%Y-%m-%d').date()).days
        pnl = round(i[4],2)
        stk_width = i[-1] - i[-2]

    update_close = [(entry_date, exit_date, days_in_trade,ticker,stk_width,pnl)]

    sql2 = """
        INSERT INTO TRADE_SUMMARY( 
        entry_date,
        exit_date,
        trade_length,
        ticker,
        stk_width,
        pnl
        ) VALUES (julianday(?),julianday(?),?,?,?,?)
    """
     
    sql3 = f"""

    INSERT INTO TRADE_HIST 
    SELECT * FROM TRADE_MAIN
    WHERE trade_group_id = '{uuid}'

    """

    sql4 = f"""

        DELETE FROM TRADE_MAIN
        WHERE trade_group_id = '{uuid}'

    """

    with con:
        con.executemany(sql2, update_close)
        con.execute(sql3)
        con.execute(sql4)

In [11]:
entry1 = create_new_entry("2021-02-09", "AAPL", 127.5, 0, "2021-03-19", 130, 0, "2021-03-19", 73.75, 0.80)
entry2 = create_new_entry("2021-02-03", "SQ", 210, 0, "2021-03-19", 212.5, 0, "2021-03-19", 78.75, 0.80)

In [12]:
enter(entry1)
enter(entry2)

In [15]:
get_stats('b48e3425-bde7-4485-b1df-d03f1609be5a')

AAPL -67 128.4 1.6 1.52 0.8 152.5


In [16]:
roll1 = update_trade('b48e3425-bde7-4485-b1df-d03f1609be5a', "2021-02-24", "AAPL", 127.5, 0, "2021-03-26", 130, 0 , "2021-03-26", 34.12, 0.37)
roll2 = update_trade('b48e3425-bde7-4485-b1df-d03f1609be5a', "2021-03-16", "AAPL", 120.0, 0, "2021-05-21", 125.0, 0 ,"2021-05-21", 7.16, 0.10)
roll3 = update_trade('b48e3425-bde7-4485-b1df-d03f1609be5a', "2021-04-09", "AAPL", 120.0, 0, "2021-05-21", 125.0, 0 ,"2021-05-21", -110.42, 1.11)

In [4]:
data = con.execute("""
    SELECT * FROM TRADE_MAIN
""")

options = []

for i in data:
    options.append({'label':i[3], 'value':i[1]})

In [16]:
def update_trade(uuid, entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice, strategy):
    """
    This function is to help you prepare trade information in the right format for writing to the database
    """
    entry = [
    (uuid, entry_date, ticker, stk1, s1t, s1e, stk2, s2t, s2e, cost, trdprice, (((stk2-stk1)*100)-cost), strategy)
]
    return entry


# function for entering trade
def enter(dbcon, new_entryl):
    """
    This function directly enters the information into the database
    """
    sql = """
        INSERT INTO TRADE_MAIN(
        trade_group_id, 
        entry_date,
        ticker,
        stk1 ,
        stk1_type,
        stk1_exp,
        stk2,
        stk2_type,
        stk2_exp,
        stk3 ,
        stk3_type,
        stk3_exp,
        stk4,
        stk4_type,
        stk4_exp,
        cost,
        trdprice,
        BRP,
        strategy
        ) VALUES (?, julianday(?),?,?,?,julianday(?),?,?,julianday(?),?,?,julianday(?),?,?,julianday(?),?,?,?,?)

    """
    with dbcon:
        dbcon.executemany(sql,new_entryl)

In [34]:
clean = pd.read_csv("opentrades.csv").drop("id", axis=1)

In [35]:
insertion = list(clean.itertuples(index=False, name=None))

In [36]:
enter(con, insertion)

In [6]:
sql = """
    INSERT INTO TRADE_SUMMARY( 
    entry_date,
    exit_date,
    trade_length,
    ticker,
    stk_width,
    pnl,
    strategy
    ) VALUES (julianday(?),julianday(?),?,?,?,?,?)

"""

In [7]:
with con:
    con.executemany(sql, insertion)

In [3]:
import plotly.express as px
import datetime
import julian
import numpy as np

In [4]:
sql  = """

SELECT julianday(exit_date), pnl FROM TRADE_SUMMARY ORDER BY exit_date

"""

In [5]:
data = con.execute(sql)

dates = []
pnl = []

for i in data:
    dates.append(julian.from_jd(i[0], fmt='jd').date())
    pnl.append(i[1])

In [70]:
data2 = con.execute(f"""
    SELECT ticker, date(entry_date), sum(trdprice), sum(cost), stk1, stk2, stk3, stk4, stk1_type FROM TRADE_MAIN
    GROUP BY trade_group_id
""")

ticker = []
start_date = []
be_price = []
cost = []
stk1 = []
stk2 = []
stk3 = []
stk4 = []
be50 = []

for i in data2:
    ticker.append(i[0])
    start_date.append(i[1])
    cost.append(round(i[3]/100,2))
    be50.append(round(i[3]/100/2,2))
    stk1.append(i[4])
    stk2.append(i[5])
    stk3.append(i[6])
    stk4.append(i[7])
    if (i[6] == None) and (i[8] == 1) and (i[5] != None):
        be_price.append(round((min(i[4], i[5])-i[3]/100),2))
    elif (i[6] == None) and (i[5] == None) and (i[8] == 0):
        be_price.append(round(i[4]-i[3]/100,2))
    elif (i[6] == None) and (i[5] == None) and (i[8] == 1):
        be_price.append(round(i[4]-i[3]/100,2))
    elif (i[6] == None) and (i[8] == 0) and (i[5] != None):
        be_price.append(round((max(i[4], i[5])-i[3]/100),2))
    elif (i[6] != None):
        be_price.append(str(round((max(i[4], i[5])-i[3]/100),2))+", "+str(round((min(i[6], i[7])-i[3]/100),2)))
    else:
        be_price.append(i[8])

trade_overview = pd.DataFrame({
    "ticker": ticker,
    "date": start_date,
    "Breakeven" : cost,
    "Breakeven 50" : be50,
    # "Strike 1" : stk1,
    # "Strike 2" : stk2,
    "Breakeven Stock Price" : be_price
})

In [71]:
be_price

trade_overview

,ticker,date,Breakeven,Breakeven 50,Breakeven Stock Price
0,RIDE,2021-04-19,1.32,0.66,7.68
1,JPM,2021-03-29,1.24,0.62,143.76
2,DIS,2021-03-29,1.31,0.66,173.69
3,QQQ,2021-03-09,0.36,0.18,339.64
4,NOK,2021-01-27,-1.51,-0.76,11.51
5,QS,2021-04-07,1.26,0.63,38.74
6,PINS,2021-04-26,1.30,0.65,"63.7, 83.7"
7,ARKF,2021-03-09,-15.18,-7.59,60.18
8,AMD,2020-10-12,-17.56,-8.78,97.56


In [73]:
sql = "UPDATE TRADE_MAIN SET stk2_type = REPLACE(stk2_type,'P',0)"

with con:
    con.execute(sql)

In [9]:
px.line(x=dates, y=np.cumsum(pnl), 
labels={"x" : "Date", "y" : "PnL"}, template="plotly_dark")